In [3]:
import pandas as pd
import numpy as np

In [4]:
df_original = pd.read_csv("2017_Migrant_A_desensitized2017_Migrant_A_desensitized.csv", encoding="GBK")
df_original[:5]

/opt/homebrew/anaconda3/envs/biling/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (81,82,85,86,87,88,89,90,92,93,94,97,116,160,161,162,163,164,165,166,167,169,170,171,172,173,174,175,180,181,182,183,184,186,227,228,229,230,231,232,233,234,236,237,238,239,240,241,242,243,245,246,247,248,249,250,251,252,286,288,292) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,newID,C1,C2,C3,C6,C7,C14,Q100,q101id1,q101a1,...,Q504C2,Q504D1,Q504D2,Q504E1,Q504E2,Q505,Q506,pcode,hjpro,w_l
0,105603,广东省,深圳市,光明新区,5281,居委会,5281,4,1,本人,...,本地,否,NaN,否,NaN,已经办理,1,44,36,6.719342
1,105619,广东省,深圳市,光明新区,5281,居委会,5281,3,1,本人,...,NaN,是,本地,否,NaN,已经办理,1,44,36,10.147591
2,73661,福建省,厦门市,同安区,3684,居委会,3684,4,1,本人,...,NaN,是,本地,否,NaN,已经办理,1,35,51,1.355456
3,168796,新疆生产建设兵团,第六师,军户农场,8440,居委会,8440,3,1,本人,...,本地,否,NaN,否,NaN,已经办理,2,66,65,0.157172
4,1718,北京市,北京市,朝阳区,86,居委会,86,2,1,本人,...,NaN,是,本地,否,NaN,已经办理,1,11,51,1.585694


In [14]:
df = df_original

In [15]:
print(", ".join(df.columns))

newID, C1, C2, C3, C6, C7, C14, Q100, q101id1, q101a1, q101b1, q101c1y, q101c1m, q101d1, q101e1, q101f1, q101g1, q101h1, q101i1, q101j1a, q101j1b, q101k1, q101l1, q101m1y, q101m1m, q101n1, q101id2, q101a2, q101b2, q101c2y, q101c2m, q101d2, q101e2, q101f2, q101g2, q101h2, q101i2, q101j2a, q101j2b, q101k2, q101l2, q101m2y, q101m2m, q101n2, q101id3, q101a3, q101b3, q101c3y, q101c3m, q101d3, q101e3, q101f3, q101g3, q101h3, q101i3, q101j3a, q101j3b, q101k3, q101l3, q101m3y, q101m3m, q101n3, q101id4, q101a4, q101b4, q101c4y, q101c4m, q101d4, q101e4, q101f4, q101g4, q101h4, q101i4, q101j4a, q101j4b, q101k4, q101l4, q101m4y, q101m4m, q101n4, q101id5, q101a5, q101b5, q101c5y, q101c5m, q101d5, q101e5, q101f5, q101g5, q101h5, q101i5, q101j5a, q101j5b, q101k5, q101l5, q101m5y, q101m5m, q101n5, q101id6, Q102, Q102A, Q102B, Q103, Q104, Q105, Q106, Q106A, Q106B, Q107, Q108A, Q108B, Q109, Q109A, Q110, Q110A, Q111, Q112, Q201, Q201A, Q202, Q203Y, Q203M, Q204, Q205, Q206, Q207, Q208, Q209, Q210Y, Q210M,

In [16]:
df["Q502"] = df["Q502A"]+df["Q502B"]+df["Q502C"]+df["Q502D"]+df["Q502E"]
def map502(x):
    if "经常" in x:
        return 4
    elif "有时" in x:
        return 3
    elif "偶尔" in x:
        return 2
    elif "没有" in x:
        return 1
    else:
        return np.nan
df["Q502"] = df["Q502"].apply(lambda x: map502(x))

#### Filling out the forms by themselves?

In [17]:
df["q101i1"].unique()

array(['否'], dtype=object)

#### Keep only selected attributes and replace strings with values

In [19]:
df = df[["Q501A","Q501B","Q501C","Q501D","Q501E","Q501F",
         'q101e1',"q101k1","q101l1","q101b1",'q101d1','q101h1', 'q101l1',
         "Q104",'Q105',
         'Q201', "Q208",
         'Q305A','Q305B','Q305C','Q305D','Q305E','Q305F','Q305G','Q305H',
         "Q309", "Q313", "Q314",
         'Q406','Q410',
         "Q502",
         "Q503A","Q503D","Q503E", "Q503F", "Q503H",
         'Q504A1','Q504A2','Q504B1','Q504B2','Q504C1','Q504C2','Q504D1','Q504D2','Q504E1','Q504E2']]

In [20]:
df.Q309.unique()

array(['其他外地人', '其他本地人', '同乡（户口仍在老家）', '很少与人来往', '同乡（户口迁至本地）',
       '同乡（户口迁至本地与老家以来的其他地区）', '21'], dtype=object)

In [21]:
import numpy as np
d = {"经常": 4,
     "有时": 3,
     "偶尔": 2,
     "没有":1,
     '完全同意':2,
     '基本同意':1,
     '不同意':-1,
     '完全不同意':-2,
     "不清楚": np.nan,
     "女":"female",
     "男":"male",
     '未上过小学':"no education",
     '小学':"primary school",
     '初中':"midschool", 
     '高中/中专':"highschool",
     '大学专科':"junior college",
     '大学本科':"college",
     '研究生':"grad",
     '女':1,
     '男':2,
     '汉':1,
     '回':2,
     '蒙':3, 
     '彝':4, 
     '土家':5, 
     '苗':6, 
     '白':7, 
     '藏':8, 
     '侗':9,
     '其他':"other", 
     '满':10, 
     '朝鲜':11,
     '黎':12, 
     '维吾尔':13, 
     '哈萨克':14, 
     '哈尼':15, 
     '傣':16, 
     '布依':17, 
     '瑶':18,
     '未婚':"unmarried",
     '同居':"living together",
     '初婚':"just married",
     '再婚':"remarried", 
     '离婚':"divorced",
     '丧偶':"widowed",
     '均未患有':0, 
     '未就诊':np.nan, 
     '患有糖尿病':"db", 
     '患有高血压和糖尿病':"db+hpt", 
     '患有高血压':"hpt",
     '健康': 4, 
     '基本健康': 3, 
     '不健康，但生活能自理': 2, 
     '生活不能自理': 1,
     '是，最近一次发生在两周前': "within 2 weeks", 
     '是，最近一次发生在两周内': "before 2 weeks",
     '跨省':"interstate",
     '省内跨市':"intercity", 
     '市内跨县':"intercounty",
     '有固定雇主的雇员':"stable job",
     '雇主':"stable job", 
     '自营劳动者':"stable job", 
     '其他':"other",
     '无固定雇主的雇员（零工、散工等）':"unstable job",
     '愿意':1, 
     '不愿意':-1, 
     '是':1,
     '没想好':0, 
     '否':-1,
     '其他外地人': "with other foreiners", 
     '其他本地人': "with other natives",
     '很少与人来往': "don't hangout often",
     '同乡（户口仍在老家）': "people from same place", 
     '同乡（户口迁至本地）': "people from same place",
     '同乡（户口迁至本地与老家以来的其他地区）': "people from same place"
    }

In [22]:
def replace(x):
    for k,v in d.items():
        if x == k:
            return v
    return x
df=df.applymap(lambda x: replace(x))
df[:5]

,Q501A,Q501B,Q501C,Q501D,Q501E,Q501F,q101e1,q101k1,q101l1,q101b1,...,Q504A1,Q504A2,Q504B1,Q504B2,Q504C1,Q504C2,Q504D1,Q504D2,Q504E1,Q504E2
0,-1,-1,-1,-1,-1,-1,highschool,本地,interstate,1,...,-1.0,NaN,-1.0,NaN,1.0,本地,-1,NaN,-1.0,NaN
1,-1,-1,-1,-1,-1,-1,junior college,本地,interstate,1,...,-1.0,NaN,NaN,NaN,NaN,NaN,1,本地,-1.0,NaN
2,-1,-1,1,-1,-1,1,highschool,本地,interstate,1,...,-1.0,NaN,-1.0,NaN,-1.0,NaN,1,本地,-1.0,NaN
3,-1,-1,1,-1,-1,-1,midschool,本地,intercity,2,...,-1.0,NaN,-1.0,NaN,1.0,本地,-1,NaN,-1.0,NaN
4,-1,-1,-1,-1,-1,-1,junior college,本地,interstate,1,...,-1.0,NaN,-1.0,NaN,-1.0,NaN,1,本地,-1.0,NaN


In [23]:
df[:5]

,Q501A,Q501B,Q501C,Q501D,Q501E,Q501F,q101e1,q101k1,q101l1,q101b1,...,Q504A1,Q504A2,Q504B1,Q504B2,Q504C1,Q504C2,Q504D1,Q504D2,Q504E1,Q504E2
0,-1,-1,-1,-1,-1,-1,highschool,本地,interstate,1,...,-1.0,NaN,-1.0,NaN,1.0,本地,-1,NaN,-1.0,NaN
1,-1,-1,-1,-1,-1,-1,junior college,本地,interstate,1,...,-1.0,NaN,NaN,NaN,NaN,NaN,1,本地,-1.0,NaN
2,-1,-1,1,-1,-1,1,highschool,本地,interstate,1,...,-1.0,NaN,-1.0,NaN,-1.0,NaN,1,本地,-1.0,NaN
3,-1,-1,1,-1,-1,-1,midschool,本地,intercity,2,...,-1.0,NaN,-1.0,NaN,1.0,本地,-1,NaN,-1.0,NaN
4,-1,-1,-1,-1,-1,-1,junior college,本地,interstate,1,...,-1.0,NaN,-1.0,NaN,-1.0,NaN,1,本地,-1.0,NaN


#### Preselection of the study cohort
1. Ethnicity: Han
2. Not injured in the last 2 weeks
3. Migrated by themselves

In [24]:
df=df[(df.q101d1==1)&(df.Q305A==1)&(df.Q410==-1)]
len(df)

33697

In [25]:
print(df["q101k1"].unique()) 
df=df.drop("q101k1",axis=1)
df["q101b1"] = df_original["q101b1"]
print(df["q101b1"].unique())

def sex(x):
    if x=="女":
        return "female" 
    elif x=="男":
        return "male"
    else:
        return np.nan
df["q101b1"] = df["q101b1"].map(lambda x: sex(x))
df["q101b1"]

['本地']
['女' '男']


0         female
2         female
4         female
16          male
18        female
           ...  
169962    female
169968      male
169973      male
169977      male
169985      male
Name: q101b1, Length: 33697, dtype: object

#### Deal with bounding and bridging

In [26]:
df["Q501_bridging"] = df.transpose()[:2].sum().transpose().apply(lambda x: 1 if x>0 else 0)
df["Q501_bounding"] = df.transpose()[2:5].sum().transpose().apply(lambda x: 1 if x>0 else 0)
df["Q501_other"] = df.transpose()[5:6].sum().transpose().apply(lambda x: 1 if x>0 else 0)

In [27]:
df["Q501"] = [" "]*len(df)
df["Q501"] [(df.Q501_bridging==1)&(df.Q501_bounding==1)] = "both"
df["Q501"] [(df.Q501==" ")&(df.Q501_bounding==1)] = "bounding"
df["Q501"] [(df.Q501==" ")&(df.Q501_bridging==1)] = "bridging"
df["Q501"] [(df.Q501==" ")&(df.Q501_other==1)] = "other"
df["Q501"] = df["Q501"].replace(" ","no group")
df=df.drop(["Q501_bridging","Q501_bounding","Q501_other"],axis=1)

/opt/homebrew/anaconda3/envs/biling/lib/python3.8/site-packages/pandas/core/generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [28]:
df = df.drop(["Q501A","Q501B","Q501C","Q501D","Q501E","Q501F"], axis=1)

In [29]:
df["Q504"] = df['Q504A1'].astype(float)+df['Q504B1'].astype(float)+df['Q504C1'].astype(float)+df['Q504D1'].astype(float)+df['Q504E1'].astype(float)
df["Q504"] = df["Q504"].apply(lambda x: 1 if x>0 else 0).astype(int)
df = df.drop([ 'Q504A1','Q504A2','Q504B1','Q504B2','Q504C1','Q504C2','Q504D1','Q504D2','Q504E1','Q504E2'],axis=1)
df[:5]

,q101e1,q101l1,q101b1,q101d1,q101h1,q101l1,Q104,Q105,Q201,Q208,...,Q406,Q410,Q502,Q503A,Q503D,Q503E,Q503F,Q503H,Q501,Q504
0,highschool,interstate,female,1,just married,interstate,10000.0,NaN,1,stable job,...,0,-1,2,1,-1,1,-1,-1,no group,0
2,highschool,interstate,female,1,just married,interstate,40000.0,NaN,1,stable job,...,0,-1,2,2,1,1,1,2,no group,0
4,junior college,interstate,female,1,just married,interstate,9000.0,NaN,1,stable job,...,0,-1,1,1,1,-2,1,1,no group,0
16,highschool,interstate,male,1,just married,interstate,2000.0,-10000.0,-1,NaN,...,0,-1,4,2,1,-2,1,1,no group,0
18,midschool,intercounty,female,1,just married,intercounty,5000.0,-8000.0,1,stable job,...,0,-1,2,2,2,-2,-2,-1,no group,0


In [30]:
# df["Q305_type1"] = df[['Q305A']].replace(np.nan,0)
# df["Q305_type1"].unique()



In [31]:
# df["q305_2"]=df[['Q305B','Q305C','Q305D','Q305E','Q305F']].replace(np.nan,0).sum(axis=1)
# df["Q305_type2"] = df["q305_2"].apply(lambda x: 1 if x>0 else 0)
# df["Q305_type2"].unique()


In [32]:
# df["q305_2"]=df[['Q305G','Q305H']].replace(np.nan,0).sum(axis=1)
# df["Q305_type3"] = df["q305_2"].apply(lambda x: 1 if x>0 else 0)
# df["Q305_type3"].unique()
# df = df.drop("q305_2",axis=1)

In [33]:
df=df.drop(['Q305A','Q305B','Q305C','Q305D','Q305E','Q305F','Q305G','Q305H'],axis=1)

In [34]:
df.sample(10)

,q101e1,q101l1,q101b1,q101d1,q101h1,q101l1,Q104,Q105,Q201,Q208,...,Q406,Q410,Q502,Q503A,Q503D,Q503E,Q503F,Q503H,Q501,Q504
155765,highschool,intercity,male,1,just married,intercity,4000.0,13000.0,1,stable job,...,0,-1,1,1,-1,-1,2,-2,no group,0
145428,college,intercounty,female,1,just married,intercounty,6000.0,10000.0,1,stable job,...,0,-1,1,1,2,-1,1,2,no group,0
5748,midschool,interstate,male,1,just married,interstate,1000.0,2000.0,1,stable job,...,0,-1,3,1,1,1,1,1,no group,0
134179,primary school,interstate,female,1,just married,interstate,3000.0,9000.0,1,stable job,...,0,-1,2,1,-1,-1,-2,2,no group,0
146066,midschool,interstate,male,1,just married,interstate,4000.0,10000.0,1,stable job,...,0,-1,1,1,2,-2,-1,-1,no group,0
98192,highschool,intercity,male,1,just married,intercity,2000.0,6000.0,1,stable job,...,0,-1,1,1,1,-1,-1,1,no group,0
110959,midschool,intercity,male,1,just married,intercity,3500.0,7000.0,1,stable job,...,0,-1,1,1,1,1,1,1,no group,0
57768,highschool,interstate,male,1,unmarried,interstate,3300.0,5000.0,1,stable job,...,0,-1,1,2,2,-1,1,2,no group,0
91278,highschool,intercity,male,1,just married,intercity,5000.0,6000.0,1,stable job,...,0,-1,2,2,2,-2,-2,2,no group,0
130328,midschool,intercounty,female,1,just married,intercounty,333.0,8333.0,1,stable job,...,0,-1,1,2,-1,-1,2,2,no group,0


In [35]:
df = df.reset_index()
df.insert(0,"participant",df["index"].astype(str))
df=df.drop("index",axis=1)
df[:5]

,participant,q101e1,q101l1,q101b1,q101d1,q101h1,q101l1,Q104,Q105,Q201,...,Q406,Q410,Q502,Q503A,Q503D,Q503E,Q503F,Q503H,Q501,Q504
0,0,highschool,interstate,female,1,just married,interstate,10000.0,NaN,1,...,0,-1,2,1,-1,1,-1,-1,no group,0
1,2,highschool,interstate,female,1,just married,interstate,40000.0,NaN,1,...,0,-1,2,2,1,1,1,2,no group,0
2,4,junior college,interstate,female,1,just married,interstate,9000.0,NaN,1,...,0,-1,1,1,1,-2,1,1,no group,0
3,16,highschool,interstate,male,1,just married,interstate,2000.0,-10000.0,-1,...,0,-1,4,2,1,-2,1,1,no group,0
4,18,midschool,intercounty,female,1,just married,intercounty,5000.0,-8000.0,1,...,0,-1,2,2,2,-2,-2,-1,no group,0


In [36]:
df["Q401"] = df_original["Q401"]
d = dict(zip(['健康', '基本健康', '不健康，但生活能自理', '生活不能自理'],[4,3,2,1]))
for k,v in d.items():
    df["Q401"] = df["Q401"].replace(k,v)
df.groupby("Q401").agg("count")

,participant,q101e1,q101l1,q101b1,q101d1,q101h1,q101l1,Q104,Q105,Q201,...,Q406,Q410,Q502,Q503A,Q503D,Q503E,Q503F,Q503H,Q501,Q504
Q401,,,,,,,,,,,,,,,,,,,,,
1,87,87,87,87,87,87,87,87,87,87,...,86,87,87,87,87,87,87,87,87,87
2,2055,2055,2055,2055,2055,2055,2055,2054,2055,2055,...,2003,2055,2055,2055,2055,2055,2055,2055,2055,2055
3,6158,6158,6158,6158,6158,6158,6158,6158,6157,6158,...,6025,6158,6158,6158,6158,6158,6158,6158,6158,6158
4,25397,25397,25397,25397,25397,25397,25397,25394,25395,25397,...,24918,25397,25397,25397,25397,25397,25397,25397,25397,25397


In [37]:
df=df.drop(['Q410'],axis=1)

In [38]:
columns_names = {"participant": "participant",
                 "Q502":"participated_in_group_activity",
                 "Q503A":"like.current.city",
                 "Q503D": "natives.like.me",
                 "Q503E": "natives.lookdown.me",
                 "Q503F": "previous.customs.better",
                 "Q503H": "i.am.native",
                 "Q104": "expence",
                 "Q105": "income",
                 "q101e1": "education",
                 "q101b1": "gender",
                 "q101d1": "ethnicity",
                 "q101h1": "marriage",
                 "q101l1": "migration.scale",
                 "Q201": "worked_before5.1",
                 "Q208": "job",
                 "Q309": "hangouts",
                 "Q313": "willing.to.movein",
                 "Q314": "willing.to.stay",
                 "Q406": "diabete.or.hypertension",
                 "Q401": "health",
                 "Q501": "group",
                 "Q504": "insuranced"}
df.columns = [columns_names[ID] for ID in list(df.columns)]

In [39]:
df.to_csv("preliminary_data.csv",index=False)


In [41]:
print(" ".join(df.columns))


participant education migration.scale gender ethnicity marriage migration.scale expence income worked_before5.1 job hangouts willing.to.movein willing.to.stay diabete.or.hypertension participated_in_group_activity like.current.city natives.like.me natives.lookdown.me previous.customs.better i.am.native group insuranced health


## processed

In [194]:
import pandas as pd
predat = pd.read_csv("processed_data.csv")
predat

,participant,like.current.city,natives.like.me,natives.lookdown.me,native.customs.better,i.am.native,income,education,gender,ethnicity,...,willing.to.movein,willing.to.stay,migration.type,money.left,natives_inclusion,city_inclusion,tendency.livehere,migration.companionship,education.group,ethnicity.group
0,0.0,1,-1,1,-1,-1,NaN,highschool,female,1,...,1,1,self,NaN,-2,1,2,-1,NaN,han
1,1.0,2,1,1,-1,2,NaN,junior college,female,1,...,1,1,relatives,NaN,0,5,2,1,NaN,han
2,2.0,2,1,1,1,2,NaN,highschool,female,1,...,1,1,self,NaN,0,3,2,-1,NaN,han
3,3.0,1,1,-1,-1,2,NaN,midschool,male,1,...,-1,1,relatives,NaN,2,4,0,1,NaN,han
4,4.0,1,1,-2,1,1,NaN,junior college,female,1,...,1,1,self,NaN,3,1,2,-1,NaN,han
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169984,169984.0,2,2,-2,-2,2,100000.0,primary school,female,1,...,-1,1,relatives,90000.0,4,6,0,1,NaN,han
169985,169985.0,2,2,-2,1,1,120000.0,midschool,male,1,...,-1,1,self,105000.0,4,2,0,-1,NaN,han
169986,169986.0,2,2,-1,1,2,180000.0,college,female,10,...,1,1,self,60000.0,3,3,2,-1,NaN,other
169987,169987.0,2,2,-1,1,2,200000.0,midschool,female,1,...,1,1,self,100000.0,3,3,2,-1,NaN,han


In [78]:
columns = df.columns
len(columns)

18

In [259]:
df[df.group=="no_group"]

,participant,like.current.city,natives.like.me,natives.lookdown.me,native.customs.better,i.am.native,income,education,gender,ethnicity,...,group,participated_in_group_activity,insuranced,migration.self,migration.relatives,migration.fellows,health,migration.type,income.normalized,income.thousand
0,0.0,1,-1,1,-1,-1,NaN,3,1,1,...,no_group,2,1,1,0,0,4,self,NaN,NaN
1,1.0,2,1,1,-1,2,NaN,4,1,1,...,no_group,1,0,0,1,0,3,relatives,NaN,NaN
4,4.0,1,1,-2,1,1,NaN,4,1,1,...,no_group,1,1,1,0,0,4,self,NaN,NaN
11,11.0,2,1,-1,-1,2,-20000.0,3,1,1,...,no_group,2,1,0,1,1,3,relatives,-4.711765,-20.0
14,14.0,1,1,1,1,-2,-12500.0,2,2,1,...,no_group,2,1,0,1,1,4,relatives,-3.409485,-12.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169973,169973.0,1,-1,1,-2,1,90000.0,2,2,1,...,no_group,1,1,1,0,0,4,self,14.388329,90.0
169974,169974.0,1,1,2,-1,-1,90000.0,3,2,1,...,no_group,1,0,1,0,0,3,self,14.388329,90.0
169977,169977.0,2,-1,-1,-1,1,99000.0,5,2,1,...,no_group,2,1,1,0,0,4,self,15.951064,99.0
169984,169984.0,2,2,-2,-2,2,100000.0,1,1,1,...,no_group,1,0,0,1,0,4,relatives,16.124701,100.0


In [260]:
df[df.group=="both"]

,participant,like.current.city,natives.like.me,natives.lookdown.me,native.customs.better,i.am.native,income,education,gender,ethnicity,...,group,participated_in_group_activity,insuranced,migration.self,migration.relatives,migration.fellows,health,migration.type,income.normalized,income.thousand
44,44.0,2,1,-1,1,1,-2750.0,2,2,1,...,both,2,1,1,0,0,4,self,-1.716523,-2.75
130,130.0,2,2,-2,-2,2,0.0,1,2,2,...,both,2,1,0,1,0,2,relatives,-1.239020,0.00
169,169.0,2,2,-1,-1,1,0.0,3,1,1,...,both,2,1,1,0,0,4,self,-1.239020,0.00
177,177.0,2,2,-1,-1,1,0.0,5,2,1,...,both,2,1,1,0,0,4,self,-1.239020,0.00
196,196.0,1,1,-2,-1,1,0.0,5,1,1,...,both,2,1,0,1,0,4,relatives,-1.239020,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169956,169956.0,2,2,-2,-1,-1,90000.0,5,2,1,...,both,3,1,1,0,0,4,self,14.388329,90.00
169960,169960.0,1,1,-1,-1,1,90000.0,6,2,1,...,both,4,0,1,0,0,4,self,14.388329,90.00
169962,169962.0,1,1,-1,-1,2,90000.0,3,1,1,...,both,2,1,1,0,0,4,self,14.388329,90.00
169978,169978.0,2,2,-1,-1,-1,100000.0,6,2,1,...,both,2,1,1,0,0,4,self,16.124701,100.00


In [261]:
print(" + ".join(df.columns))

participant + like.current.city + natives.like.me + natives.lookdown.me + native.customs.better + i.am.native + income + education + gender + ethnicity + marriage + worked_before5.1 + diabete_hypertension + recent_disease + group + participated_in_group_activity + insuranced + migration.self + migration.relatives + migration.fellows + health + migration.type + income.normalized + income.thousand


In [10]:
for k,v in d.items():
    df=df.replace(k,v)
df

,participant,Q503A,Q503D,Q503E,Q503F,Q503H,Q105,q101e1,q101b1,q101d1,...,Q201,Q406,Q410,Q501,Q502,Q504,Q305_type1,Q305_type2,Q305_type3,Q401
0,0,1,-1,1,-1,-1,NaN,3,1,1,...,1,0.0,0,,2,1,1,0,0,1
1,1,2,1,1,-1,2,NaN,4,1,1,...,0,0.0,0,,1,0,0,1,0,2
2,2,2,1,1,1,2,NaN,3,1,1,...,1,0.0,0,bounding,2,1,1,0,0,1
3,3,1,1,-1,-1,2,NaN,2,2,1,...,1,0.0,0,bounding,2,1,0,1,0,1
4,4,1,1,-2,1,1,NaN,4,1,1,...,1,0.0,0,,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169984,169984,2,2,-2,-2,2,100000.0,1,1,1,...,0,0.0,0,,1,0,0,1,0,1
169985,169985,2,2,-2,1,1,120000.0,2,2,1,...,1,2.0,0,,2,1,1,0,0,1
169986,169986,2,2,-1,1,2,180000.0,5,1,10,...,1,0.0,1,bounding,2,1,1,0,0,2
169987,169987,2,2,-1,1,2,200000.0,2,1,1,...,1,0.0,1,bounding,4,1,1,0,0,1


In [198]:
instruction = pd.DataFrame.from_dict(d,orient='index')
instruction.columns = ["value"]
instruction

,value
经常,4
有时,3
偶尔,2
没有,1
完全同意,2
...,...
其他本地人,with other natives
很少与人来往,don't hangout often
同乡（户口仍在老家）,people from same place
同乡（户口迁至本地）,people from same place


In [199]:
instruction.to_csv("instruction.csv")


In [205]:
print("\n".join(df_original.columns))

newID
C1
C2
C3
C6
C7
C14
Q100
q101id1
q101a1
q101b1
q101c1y
q101c1m
q101d1
q101e1
q101f1
q101g1
q101h1
q101i1
q101j1a
q101j1b
q101k1
q101l1
q101m1y
q101m1m
q101n1
q101id2
q101a2
q101b2
q101c2y
q101c2m
q101d2
q101e2
q101f2
q101g2
q101h2
q101i2
q101j2a
q101j2b
q101k2
q101l2
q101m2y
q101m2m
q101n2
q101id3
q101a3
q101b3
q101c3y
q101c3m
q101d3
q101e3
q101f3
q101g3
q101h3
q101i3
q101j3a
q101j3b
q101k3
q101l3
q101m3y
q101m3m
q101n3
q101id4
q101a4
q101b4
q101c4y
q101c4m
q101d4
q101e4
q101f4
q101g4
q101h4
q101i4
q101j4a
q101j4b
q101k4
q101l4
q101m4y
q101m4m
q101n4
q101id5
q101a5
q101b5
q101c5y
q101c5m
q101d5
q101e5
q101f5
q101g5
q101h5
q101i5
q101j5a
q101j5b
q101k5
q101l5
q101m5y
q101m5m
q101n5
q101id6
Q102
Q102A
Q102B
Q103
Q104
Q105
Q106
Q106A
Q106B
Q107
Q108A
Q108B
Q109
Q109A
Q110
Q110A
Q111
Q112
Q201
Q201A
Q202
Q203Y
Q203M
Q204
Q205
Q206
Q207
Q208
Q209
Q210Y
Q210M
Q211
Q212
Q212A
Q213Y
Q213M
Q214
Q215
Q216
Q301
Q302Y
Q302M
Q303
Q304A
Q304B
Q304D
Q305A
Q305B
Q305C
Q305D
Q305E
Q305F
Q305G
Q305

In [302]:
## more variables to study

comp = df_original[["q101l1","Q104","Q208","Q313","Q314"]]
comp


,q101l1,Q104,Q208,Q313,Q314
0,跨省,10000.0,有固定雇主的雇员,愿意,是
1,跨省,60000.0,NaN,愿意,是
2,跨省,40000.0,雇主,愿意,是
3,省内跨市,5000.0,自营劳动者,不愿意,是
4,跨省,9000.0,有固定雇主的雇员,愿意,是
...,...,...,...,...,...
169984,跨省,10000.0,NaN,不愿意,是
169985,市内跨县,15000.0,雇主,不愿意,是
169986,跨省,120000.0,有固定雇主的雇员,愿意,是
169987,省内跨市,100000.0,雇主,愿意,是


In [303]:
comp = comp.applymap(lambda x:replace(x))

In [304]:
comp

,q101l1,Q104,Q208,Q313,Q314
0,interstate,10000.0,stable job,1,1
1,interstate,60000.0,NaN,1,1
2,interstate,40000.0,stable job,1,1
3,intercity,5000.0,stable job,-1,1
4,interstate,9000.0,stable job,1,1
...,...,...,...,...,...
169984,interstate,10000.0,NaN,-1,1
169985,intercounty,15000.0,stable job,-1,1
169986,interstate,120000.0,stable job,1,1
169987,intercity,100000.0,stable job,1,1


In [305]:
comp.columns = ["migration_scale","expence","job","willing.to.movein","willing.to.stay"]

In [306]:
df = pd.concat([df.T,comp.T]).T

In [307]:
df

,participant,like.current.city,natives.like.me,natives.lookdown.me,native.customs.better,i.am.native,income,education,gender,ethnicity,...,insuranced,migration.self,migration.relatives,migration.fellows,health,migration_scale,expence,job,willing.to.movein,willing.to.stay
0,0,1,-1,1,-1,-1,NaN,highschool,female,1,...,1,1,0,0,4,interstate,10000.0,stable job,1,1
1,1,2,1,1,-1,2,NaN,junior college,female,1,...,0,0,1,0,3,interstate,60000.0,NaN,1,1
2,2,2,1,1,1,2,NaN,highschool,female,1,...,1,1,0,0,4,interstate,40000.0,stable job,1,1
3,3,1,1,-1,-1,2,NaN,midschool,male,1,...,1,0,1,0,4,intercity,5000.0,stable job,-1,1
4,4,1,1,-2,1,1,NaN,junior college,female,1,...,1,1,0,0,4,interstate,9000.0,stable job,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169984,169984,2,2,-2,-2,2,100000.0,primary school,female,1,...,0,0,1,0,4,interstate,10000.0,NaN,-1,1
169985,169985,2,2,-2,1,1,120000.0,midschool,male,1,...,1,1,0,0,4,intercounty,15000.0,stable job,-1,1
169986,169986,2,2,-1,1,2,180000.0,college,female,10,...,1,1,0,0,3,interstate,120000.0,stable job,1,1
169987,169987,2,2,-1,1,2,200000.0,midschool,female,1,...,1,1,0,0,4,intercity,100000.0,stable job,1,1


In [308]:
df.to_csv("preliminary_data.csv",index=False)
